In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
train = pd.read_csv('train.csv')
test = pd.read_csv('test_2.csv')
result=pd.read_csv('sample_submission_2.csv')

In [2]:
print (train.shape)
print (test.shape)
print (result.shape)

(40000, 211)
(120000, 147)
(7440000, 2)


In [3]:
#fill NA
trainmed=train.fillna(train.median())
train0=train.fillna(0)
testmed=test.fillna(test.median())
test0=test.fillna(0)

In [99]:
#add features in trainmed
#calculate delta
delta=abs(trainmed.iloc[:,146]-trainmed.iloc[:,28])/119
delta=pd.DataFrame(delta,columns={'delta'})

In [77]:
#abstract Ret0 to Ret120
trtrain=np.zeros((119, 40001))
trtrain[:,0]=range(2,121)
trtrain[:,1:]=np.transpose(trainmed.iloc[:,28:147])
print trtrain.shape

(119L, 40001L)


In [94]:
#calculate drawdown duration and drawdown magnitude
drawdu=np.zeros(40000)
drawma=np.zeros(40000)
for i in range(1,40001):
    drawma[i-1]=max(trtrain[:,i])-min(trtrain[:,i])
    for j in range(0,119):
        if trtrain[j,i]==max(trtrain[:,i]):
            a=j
    for k in range(0,119):
        if trtrain[k,i]==min(trtrain[:,i]):
            b=k
    drawdu[i-1]=abs(trtrain[a,0]-trtrain[b,0])
        

In [98]:
#combine
drawdu=pd.DataFrame(drawdu,columns={'drawdown duration'})
drawma=pd.DataFrame(drawma,columns={'drawdown magnitude'})
newtrainmed = pd.concat([delta,drawdu,drawma,trainmed], axis=1)

In [104]:
newtrainmed.shape

(40000, 214)

In [ ]:
#add features in train0

In [105]:
delta0=abs(train0.iloc[:,146]-train0.iloc[:,28])/119
delta0=pd.DataFrame(delta0,columns={'delta'})
trtrain0=np.zeros((119, 40001))
trtrain0[:,0]=range(2,121)
trtrain0[:,1:]=np.transpose(train0.iloc[:,28:147])
print trtrain0.shape

(119L, 40001L)


In [107]:
#calculate drawdown duration and drawdown magnitude
drawdu0=np.zeros(40000)
drawma0=np.zeros(40000)
for i in range(1,40001):
    drawma0[i-1]=max(trtrain0[:,i])-min(trtrain0[:,i])
    for j in range(0,119):
        if trtrain0[j,i]==max(trtrain0[:,i]):
            a=j
        if trtrain0[j,i]==min(trtrain0[:,i]):
            b=j
    drawdu0[i-1]=abs(trtrain0[a,0]-trtrain0[b,0])

In [112]:
drawdu0=pd.DataFrame(drawdu0,columns={'drawdown duration'})
drawma0=pd.DataFrame(drawma0,columns={'drawdown magnitude'})
newtrain0 = pd.concat([delta0,drawdu0,drawma0,train0], axis=1)
newtrain0.shape

(40000, 214)

In [113]:
newtrain0.to_csv('newtrain0.csv')
newtrainmed.to_csv('newtrainmed.csv')

In [ ]:
#test

In [120]:
len(test0.index)

120000

In [123]:
def change(data):
    delta1=abs(data.iloc[:,146]-data.iloc[:,28])/119
    delta1=pd.DataFrame(delta1,columns={'delta'})
    trtest=np.zeros((119, len(data.index)+1))
    trtest[:,0]=range(2,121)
    trtest[:,1:]=np.transpose(data.iloc[:,28:147])
    drawdowndu=np.zeros(len(data.index))
    drawmag=np.zeros(len(data.index))
    a=0
    b=0
    for i in range(1,len(data.index)+1):
        drawmag[i-1]=max(trtest[:,i])-min(trtest[:,i])
        for j in range(0,119):
            if trtest[j,i]==max(trtest[:,i]):
                a=j
            if trtest[j,i]==min(trtest[:,i]):
                b=j
        drawdowndu[i-1]=abs(trtest[a,0]-trtest[b,0])
    drawdowndu=pd.DataFrame(drawdowndu,columns={'drawdown duration'})
    drawmag=pd.DataFrame(drawmag,columns={'drawdown magnitude'})
    newdata = pd.concat([delta1,drawdowndu,drawmag,data], axis=1)
    return newdata

In [124]:
newtest0=change(test0)

In [126]:
newtestmed=change(testmed)
newtestmed.shape

(120000, 150)

In [127]:
newtest0.to_csv('newtest0.csv')
newtestmed.to_csv('newtestmed.csv')

In [ ]:
#rescale

In [168]:
def rescalemed(data):
    from scipy.stats import skew
    del data['Id']
    skewed = data.iloc[:,0:28].apply(lambda x: skew(x.dropna().astype(float)))
    skewed = skewed[skewed > 0.75]
    skewed = skewed.index
    data[skewed] = np.log1p(data[skewed])
    data=data.fillna(data.median())
    # Additional processing: scale the data. 
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(data.iloc[:,0:28])
    scaled = scaler.transform(data.iloc[:,0:28])
    datascaled=data
    datascaled.iloc[:,0:28]=scaled
    return datascaled


In [172]:
def rescale0(data):
    from scipy.stats import skew
    del data['Id']
    skewed = data.iloc[:,0:28].apply(lambda x: skew(x.dropna().astype(float)))
    skewed = skewed[skewed > 0.75]
    skewed = skewed.index
    data[skewed] = np.log1p(data[skewed])
    data=data.fillna(0)
    # Additional processing: scale the data. 
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(data.iloc[:,0:28])
    scaled = scaler.transform(data.iloc[:,0:28])
    datascaled=data
    datascaled.iloc[:,0:28]=scaled
    return datascaled

In [169]:
strainmed=rescalemed(newtrainmed)

C:\Users\Seven\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log1p
  import sys


In [173]:
stestmed=rescalemed(newtestmed)

C:\Users\Seven\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log1p
  import sys


In [174]:
stestmed.shape

(120000, 149)

In [175]:
strain0=rescale0(newtrain0)
stest0=rescale0(newtest0)

C:\Users\Seven\Anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in log1p
  import sys


In [176]:
strain0.shape

(40000, 213)

In [180]:
strain0.to_csv('strain0.csv')
strainmed.to_csv('strainmed.csv')
stest0.to_csv('stest0.csv')
stestmed.to_csv('stestmed.csv')

In [179]:
print (strainmed.shape)
print (stestmed.shape)

(40000, 213)
(120000, 149)
